In [25]:
import os
import json

In [26]:
# load json file and read line by line
def load_json_file(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [27]:
train_path = "/research/d5/gds/xywen22/project/llm_framework/AST_abcd_part/data/processed/train_workflow_discovery_abcd_1600.json"

train_data = load_json_file(train_path)

In [28]:
print(train_data[0]['target'])

pull up the costumer account [albert sanders]; validate purchase [alberts7, alberts7@email.com, 0712999033]; record reason [guess jeans]; offer refund [174]


#### process the actions described in the format of natural language to the structured format

In [29]:
mapping_dict = {
  "pull-up-account": "pull up the costumer account",
  "enter-details": "enter details",
  "verify-identity": "verify costumer identity",
  "make-password": "create new password",
  "search-timing": "search timing",
  "search-policy": "check policy",
  "validate-purchase": "validate purchase",
  "search-faq": "search faq",
  "membership": "check membership level",
  "search-boots": "search for boots",
  "try-again": "ask costumer to try again",
  "ask-the-oracle": "ask oracle",
  "update-order": "update order information",
  "promo-code": "offer promo code",
  "update-account": "update costumer account",
  "search-membership": "get memberships information",
  "make-purchase": "make purchase",
  "offer-refund": "offer refund",
  "notify-team": "notify team",
  "record-reason": "record reason",
  "search-jeans": "search for jeans",
  "shipping-status": "get shipping status",
  "search-shirt": "search for shirt",
  "instructions": "check instructions",
  "search-jacket": "search for jacket",
  "log-out-in": "ask costumer log out log in",
  "select-faq": "select topic in faq",
  "subscription-status": "get subscription status",
  "send-link": "send link to costumer",
  "search-pricing": "check pricing"
}

# conver key to value and value to key
def convert_dict(mapping_dict):
    new_dict = {}
    for key, value in mapping_dict.items():
        new_dict[value] = key
    return new_dict

new_mapping_dict = convert_dict(mapping_dict)
print(new_mapping_dict)

{'pull up the costumer account': 'pull-up-account', 'enter details': 'enter-details', 'verify costumer identity': 'verify-identity', 'create new password': 'make-password', 'search timing': 'search-timing', 'check policy': 'search-policy', 'validate purchase': 'validate-purchase', 'search faq': 'search-faq', 'check membership level': 'membership', 'search for boots': 'search-boots', 'ask costumer to try again': 'try-again', 'ask oracle': 'ask-the-oracle', 'update order information': 'update-order', 'offer promo code': 'promo-code', 'update costumer account': 'update-account', 'get memberships information': 'search-membership', 'make purchase': 'make-purchase', 'offer refund': 'offer-refund', 'notify team': 'notify-team', 'record reason': 'record-reason', 'search for jeans': 'search-jeans', 'get shipping status': 'shipping-status', 'search for shirt': 'search-shirt', 'check instructions': 'instructions', 'search for jacket': 'search-jacket', 'ask costumer log out log in': 'log-out-in', 

In [30]:
all_transitions = []
for i in range(len(train_data)):
    # if i == 10:
    #     break
    # print(new_mapping_dict[train_data[i]['target']])
    target_pieces = train_data[i]['target'].split(';')
    # pull up the costumer account [alessandro phoenix]
    # only get the first part before the first '['
    for j in range(len(target_pieces)):
        target_pieces[j] = target_pieces[j].split('[')[0].strip()
    
    for j in range(len(target_pieces)):
        if target_pieces[j] in new_mapping_dict:
            target_pieces[j] = new_mapping_dict[target_pieces[j]]
        else:
            print("not found")

    # print(target_pieces)

    all_transitions.append(target_pieces)
print("all_transitions: ", all_transitions)

all_transitions:  [['pull-up-account', 'validate-purchase', 'record-reason', 'offer-refund'], ['pull-up-account', 'shipping-status', 'validate-purchase', 'update-order'], ['search-faq', 'search-jacket'], ['search-faq', 'search-shirt', 'select-faq'], ['search-faq', 'search-boots', 'select-faq'], ['pull-up-account', 'verify-identity', 'record-reason', 'update-account'], ['pull-up-account', 'verify-identity', 'membership', 'record-reason', 'offer-refund'], ['pull-up-account', 'record-reason', 'record-reason', 'verify-identity', 'update-account'], ['pull-up-account', 'validate-purchase', 'update-order'], ['pull-up-account', 'verify-identity', 'subscription-status', 'enter-details', 'send-link'], ['enter-details', 'log-out-in'], ['pull-up-account', 'validate-purchase'], ['offer-refund', 'pull-up-account', 'record-reason', 'ask-the-oracle', 'promo-code'], ['search-faq', 'search-policy', 'select-faq'], ['pull-up-account', 'notify-team'], ['pull-up-account', 'enter-details', 'make-password'], 

In [31]:
possible_states = mapping_dict.keys()
print(list(possible_states))
possible_states = list(possible_states)
# ['pull-up-account', 'enter-details', 'verify-identity', 'make-password', 'search-timing', 'search-policy', 'validate-purchase', 'search-faq', 'membership', 'search-boots', 'try-again', 'ask-the-oracle', 'update-order', 'promo-code', 'update-account', 'search-membership', 'make-purchase', 'offer-refund', 'notify-team', 'record-reason', 'search-jeans', 'shipping-status', 'search-shirt', 'instructions', 'search-jacket', 'log-out-in', 'select-faq', 'subscription-status', 'send-link', 'search-pricing']

['pull-up-account', 'enter-details', 'verify-identity', 'make-password', 'search-timing', 'search-policy', 'validate-purchase', 'search-faq', 'membership', 'search-boots', 'try-again', 'ask-the-oracle', 'update-order', 'promo-code', 'update-account', 'search-membership', 'make-purchase', 'offer-refund', 'notify-team', 'record-reason', 'search-jeans', 'shipping-status', 'search-shirt', 'instructions', 'search-jacket', 'log-out-in', 'select-faq', 'subscription-status', 'send-link', 'search-pricing']


In [32]:
# count the transition frequency between every two actions/state according to all_transitions
transition_count = {}
for i in range(len(all_transitions)):
    for j in range(len(all_transitions[i]) - 1):
        if (all_transitions[i][j], all_transitions[i][j+1]) in transition_count:
            transition_count[(all_transitions[i][j], all_transitions[i][j+1])] += 1
        else:
            transition_count[(all_transitions[i][j], all_transitions[i][j+1])] = 1

print("transition_count: ", transition_count)

# if no transition between two states, set the frequency to 0
for state1 in possible_states:
    for state2 in possible_states:
        if (state1, state2) not in transition_count:
            transition_count[(state1, state2)] = 0

print("transition_count: ", transition_count)

transition_count:  {('pull-up-account', 'validate-purchase'): 296, ('validate-purchase', 'record-reason'): 89, ('record-reason', 'offer-refund'): 28, ('pull-up-account', 'shipping-status'): 30, ('shipping-status', 'validate-purchase'): 32, ('validate-purchase', 'update-order'): 52, ('search-faq', 'search-jacket'): 46, ('search-faq', 'search-shirt'): 48, ('search-shirt', 'select-faq'): 34, ('search-faq', 'search-boots'): 54, ('search-boots', 'select-faq'): 38, ('pull-up-account', 'verify-identity'): 389, ('verify-identity', 'record-reason'): 9, ('record-reason', 'update-account'): 3, ('verify-identity', 'membership'): 57, ('membership', 'record-reason'): 1, ('pull-up-account', 'record-reason'): 120, ('record-reason', 'record-reason'): 11, ('record-reason', 'verify-identity'): 52, ('verify-identity', 'update-account'): 28, ('verify-identity', 'subscription-status'): 73, ('subscription-status', 'enter-details'): 28, ('enter-details', 'send-link'): 53, ('enter-details', 'log-out-in'): 10, 

In [33]:
# print the pairs number
counter = 0
for key, value in transition_count.items():
    counter += 1
    print(key, value)

print("counter: ", counter)

('pull-up-account', 'validate-purchase') 296
('validate-purchase', 'record-reason') 89
('record-reason', 'offer-refund') 28
('pull-up-account', 'shipping-status') 30
('shipping-status', 'validate-purchase') 32
('validate-purchase', 'update-order') 52
('search-faq', 'search-jacket') 46
('search-faq', 'search-shirt') 48
('search-shirt', 'select-faq') 34
('search-faq', 'search-boots') 54
('search-boots', 'select-faq') 38
('pull-up-account', 'verify-identity') 389
('verify-identity', 'record-reason') 9
('record-reason', 'update-account') 3
('verify-identity', 'membership') 57
('membership', 'record-reason') 1
('pull-up-account', 'record-reason') 120
('record-reason', 'record-reason') 11
('record-reason', 'verify-identity') 52
('verify-identity', 'update-account') 28
('verify-identity', 'subscription-status') 73
('subscription-status', 'enter-details') 28
('enter-details', 'send-link') 53
('enter-details', 'log-out-in') 10
('offer-refund', 'pull-up-account') 1
('record-reason', 'ask-the-ora

In [34]:
# count the frequency of each state
transition_matrix = {}
for state1 in possible_states:
    transition_matrix[state1] = {}
    for state2 in possible_states:
        transition_matrix[state1][state2] = 0
    
    for key, value in transition_count.items():
        if state1 == key[0]:
            transition_matrix[state1][key[1]] = value

print("transition_matrix: ", transition_matrix)
print(len(transition_matrix['pull-up-account']))

# convert count to probability
for state1 in possible_states:
    sum_count = sum(transition_matrix[state1].values())
    for state2 in possible_states:
        if sum_count != 0:
            transition_matrix[state1][state2] = transition_matrix[state1][state2] / sum_count
        
print("transition_matrix: ", transition_matrix)
print(len(transition_matrix['pull-up-account']))
print("sum of probability: ", sum(transition_matrix['pull-up-account'].values()))
print("sum of probability: ", sum(transition_matrix['enter-details'].values()))

transition_matrix:  {'pull-up-account': {'pull-up-account': 7, 'enter-details': 99, 'verify-identity': 389, 'make-password': 3, 'search-timing': 0, 'search-policy': 0, 'validate-purchase': 296, 'search-faq': 28, 'membership': 1, 'search-boots': 1, 'try-again': 7, 'ask-the-oracle': 56, 'update-order': 1, 'promo-code': 5, 'update-account': 1, 'search-membership': 0, 'make-purchase': 2, 'offer-refund': 1, 'notify-team': 19, 'record-reason': 120, 'search-jeans': 0, 'shipping-status': 30, 'search-shirt': 1, 'instructions': 0, 'search-jacket': 0, 'log-out-in': 3, 'select-faq': 0, 'subscription-status': 10, 'send-link': 0, 'search-pricing': 0}, 'enter-details': {'pull-up-account': 0, 'enter-details': 13, 'verify-identity': 2, 'make-password': 58, 'search-timing': 0, 'search-policy': 0, 'validate-purchase': 0, 'search-faq': 0, 'membership': 1, 'search-boots': 0, 'try-again': 5, 'ask-the-oracle': 0, 'update-order': 73, 'promo-code': 0, 'update-account': 34, 'search-membership': 0, 'make-purchas

In [35]:
# convert to mdp:
# here we use the dot format to represent the mdp
'''
digraph learned_mdp {
s0 [label="init"];
s1 [label="pull-up-account"];
s2 [label="end"];
s3 [label="enter-details"];
s0 -> s1  [label="pull-up-account:0.03125"];
s0 -> s3  [label="enter-details:0.03125"];
s0 -> s2  [label="end:0.03125"];
s1 -> s3  [label="pull-up-account:0.2202970297029703"];
s1 -> s1  [label="pull-up-account:0.7797029702970297"];
s1 -> s3  [label="enter-details:0.03125"];
s1 -> s2  [label="end:0.03125"];
s2 -> s1  [label="pull-up-account:0.03125"];
s2 -> s3  [label="enter-details:0.03125"];
s2 -> s2  [label="end:0.03125"];
s3 -> s1  [label="pull-up-account:0.03125"];
s3 -> s3  [label="enter-details:0.03125"];
s3 -> s2  [label="end:0.03125"];
__start0 [label="", shape=none];
__start0 -> s0  [label=""];
}
'''

# convert transition_matrix to dot format
def convert_to_dot(transition_matrix):
    dot_str = "digraph learned_mdp {\n"
    dot_str += "s0 [label=\"init\"];\n"
    for i in range(len(possible_states)):
        dot_str += "s" + str(i+1) + " [label=\"" + possible_states[i] + "\"];\n"
    dot_str += "s31 [label=\"end\"];\n"    

    dot_str += "s0 -> s0  [label=\"init:1.0\"];\n"
    dot_str += "s0 -> s1  [label=\"action:0.0343\"];\n"
    dot_str += "s0 -> s2  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s3  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s4  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s5  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s6  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s7  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s8  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s9  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s10  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s11  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s12  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s13  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s14  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s15  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s16  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s17  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s18  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s19  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s20  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s21  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s22  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s23  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s24  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s25  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s26  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s27  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s28  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s29  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s30  [label=\"action:0.0333\"];\n"
    dot_str += "s0 -> s31  [label=\"end:1\"];\n"

    for j in range(len(possible_states)):
        index = j
        state1 = possible_states[j]
        dot_str += "s" + str(index+1) + " -> s0  [label=\"init:1.0\"];\n"
        for k in range(len(possible_states)):
            index2 = k
            state2 = possible_states[k]
            if transition_matrix[state1][state2] != 0:
                dot_str += "s" + str(index+1) + " -> s" + str(index2+1) + "  [label=\"" + "action" + ":" + str(transition_matrix[state1][state2]) + "\"];\n"
        dot_str += 's' + str(index+1) + ' -> s31' + ' [label="end:1.0"];\n'
    
    dot_str += 's31 -> s31 [label="init:1.0"];\n'
    dot_str += 's31 -> s31  [label="action:1.0"];\n'
    dot_str += 's31 -> s31  [label="end:1.0"];\n'

    dot_str += "__start0 [label=\"\", shape=none];\n"
    dot_str += "__start0 -> s0  [label=\"\"];\n"
    dot_str += "}"

    return dot_str

dot_str = convert_to_dot(transition_matrix)
print(dot_str)

# write the dot string to a file
with open("learned_mdp_1600.dot", "w") as f:
    f.write(dot_str)


digraph learned_mdp {
s0 [label="init"];
s1 [label="pull-up-account"];
s2 [label="enter-details"];
s3 [label="verify-identity"];
s4 [label="make-password"];
s5 [label="search-timing"];
s6 [label="search-policy"];
s7 [label="validate-purchase"];
s8 [label="search-faq"];
s9 [label="membership"];
s10 [label="search-boots"];
s11 [label="try-again"];
s12 [label="ask-the-oracle"];
s13 [label="update-order"];
s14 [label="promo-code"];
s15 [label="update-account"];
s16 [label="search-membership"];
s17 [label="make-purchase"];
s18 [label="offer-refund"];
s19 [label="notify-team"];
s20 [label="record-reason"];
s21 [label="search-jeans"];
s22 [label="shipping-status"];
s23 [label="search-shirt"];
s24 [label="instructions"];
s25 [label="search-jacket"];
s26 [label="log-out-in"];
s27 [label="select-faq"];
s28 [label="subscription-status"];
s29 [label="send-link"];
s30 [label="search-pricing"];
s31 [label="end"];
s0 -> s0  [label="init:1.0"];
s0 -> s1  [label="action:0.0343"];
s0 -> s2  [label="acti

In [36]:
from aalpy.utils import load_automaton_from_file, save_automaton_to_file, visualize_automaton, generate_random_dfa
from aalpy.automata import Dfa
from aalpy.SULs import AutomatonSUL
from aalpy.oracles import RandomWalkEqOracle
from aalpy.learning_algs import run_Lstar, run_KV

# load an automaton
automaton = load_automaton_from_file('learned_mdp_1600.dot', automaton_type='mdp')

print(automaton)

# visualize the automaton
visualize_automaton(automaton)

digraph learnedModel {
s0 [label="init"];
s1 [label="pull-up-account"];
s2 [label="enter-details"];
s3 [label="verify-identity"];
s4 [label="make-password"];
s5 [label="search-timing"];
s6 [label="search-policy"];
s7 [label="validate-purchase"];
s8 [label="search-faq"];
s9 [label="membership"];
s10 [label="search-boots"];
s11 [label="try-again"];
s12 [label="ask-the-oracle"];
s13 [label="update-order"];
s14 [label="promo-code"];
s15 [label="update-account"];
s16 [label="search-membership"];
s17 [label="make-purchase"];
s18 [label="offer-refund"];
s19 [label="notify-team"];
s20 [label="record-reason"];
s21 [label="search-jeans"];
s22 [label="shipping-status"];
s23 [label="search-shirt"];
s24 [label="instructions"];
s25 [label="search-jacket"];
s26 [label="log-out-in"];
s27 [label="select-faq"];
s28 [label="subscription-status"];
s29 [label="send-link"];
s30 [label="search-pricing"];
s31 [label="end"];
s0 -> s0  [label="init:1.0"];
s0 -> s1  [label="action:0.03"];
s0 -> s2  [label="actio

Model saved to LearnedModel.pdf.
